## Security Analysis on LSH-based BTPS
- Seunghun Paik, Sunpill Kim and Jae Hong Seo
- BMVC 2023

## Contents
- Implementation of three LSH-based BTPs (GRP/URP-IoM [1], ABH [2])
- Implementation of the proposed attack
- Comparison with previous two attacks (Dong et al. [3], Ghammam et al. [4])

## Prerequisite
- pytorch
- matplotlib
- cvxopt (for Ghammam et al. [2])

For benchmarking of LSH-basted BTPs, three representative datasets (LFW, CFP-FP, and AgeDB are required!)

## 0. Basic Setup

In [ ]:
### Device
device = "cpu"                  ### Put your device if GPU is available

### Path for Benchmark Sets
test_dir = "your dir"

### Path for Backbone
bb_dir = "your dir"

### Path for NbNet
nb_dir = "your dir"

### Path for Demo Images
demo_dir = "./demo"

from fr_utils import get_backbone
from nbnet import get_nbnet
backbone = get_backbone("r50", bb_dir, device)
nbnet = get_nbnet(nb_dir, device)


## 1. Implementation Results of LSH-based BTPs

In [ ]:
### Parameter Preset
from BTPs import GRP, URP, ABH

grp = GRP(300, 16, 512, device)
urp = URP(600, 100, 2, 512, device)
abh = ABH(50, 60, 2, 22, 512, device)

In [ ]:
### Benchmarking
from fr_utils.verification import load_bin, test
lfw = load_bin(test_dir + "/lfw.bin", (112,112))
cfp = load_bin(test_dir + "/cfp_fp.bin", (112,112))
age = load_bin(test_dir + "/agedb_30.bin", (112,112))

### Batch Size
batch_size = 128     ### Modify this value in accordance with your experimental environment. 



### GRP-IoM

In [ ]:
print("Testing on LFW: ", test(lfw, backbone, batch_size, device))
print("Testing on CFP-FP: ", test(cfp, batch_size, device))
print("Testing on AgeDB: ", test(age, batch_size, device))

### URP-IoM

In [ ]:
print("Testing on LFW: ", test(lfw, backbone, batch_size, device))
print("Testing on CFP-FP: ", test(cfp, batch_size, device))
print("Testing on AgeDB: ", test(age, batch_size, device))

### ABH

In [ ]:
print("Testing on LFW: ", test(lfw, backbone, batch_size, device))
print("Testing on CFP-FP: ", test(cfp, batch_size, device))
print("Testing on AgeDB: ", test(age, batch_size, device))

## 2. Implementation Result of Proposed Attack

We will show the effectiveness of our attack via DEMO

In [ ]:
### Demo images in the paper
import matplotlib.pyplot as plt

### FOR ABH
img1 = plt.imread(demo_dir + "/img1.jpg")
img2 = plt.imread(demo_dir + "/img2.jpg")
### FOR GRP
img3 = plt.imread(demo_dir + "/img3.jpg")
### FOR URP
img4 = plt.imread(demo_dir + "/img4.jpg")
img5 = plt.imread(demo_dir + "/img5.jpg")

for i in range(1,6):
    plt.subplot(1, 5, i)
    plt.imshow(globals()["img" + str(i)])
    plt.axis("off")
plt.show()

### Attack Pipeline

After enrollment, the attack can be done in the following procedure:

- Step 1. FInd the pre-image of the given template
- Step 2. Using NbNet, recover the image from the pre-image.

In [ ]:
### Feature extraction
### function for pre-processing
import torch
import torch.nn.functional as F


@torch.no_grad()
def preproc_img(img):
    # Normalization 
    img = torch.tensor(img) / 127.5 - 1
    # HWC -> CHW
    img = img.permute(2,0,1)
    # Reshape
    img = F.interpolate(img.unsqueeze(0), size=(112,112))
    return img

### Tensor to Image
ten2img = lambda ten: (ten.detach().cpu().numpy().transpose(1,2,0) + 1) / 2

### Preparation
feat1 = backbone(preproc_img(img1))
feat2 = backbone(preproc_img(img2))
feat3 = backbone(preproc_img(img3))
feat4 = backbone(preproc_img(img4))
feat5 = backbone(preproc_img(img5))

### Enrollment
tmp1 = abh.hashing(feat1)
tmp2 = abh.hashing(feat2)
tmp3 = grp.hashing(feat3)
tmp4 = urp.hashing(feat4)
tmp5 = urp.hashing(feat5)


### ABH

In [ ]:
from attacks import attack_abh_efficient 

feat1_r = attack_abh_efficient(abh, tmp1)
feat2_r = attack_abh_efficient(abh, tmp2)

img1_r = nbnet(F.normalize(feat1_r))
img2_r = nbnet(F.normalize(feat2_r))

feat1_rr = backbone(F.interpolate(img1_r, (112,112)))
feat2_rr = backbone(F.interpolate(img2_r, (112,112)))

angle1 = F.cosine_similarity(feat1, feat1_rr).acos() * 180 / torch.pi
angle2 = F.cosine_similarity(feat2, feat2_rr).acos() * 180 / torch.pi


### Plotting Imgs
plt.subplot(2,2,1)
plt.title("Enrolled")
plt.imshow(img1)
plt.axis("off")

plt.subplot(2,2,2)
plt.imshow(img2)
plt.axis("off")

plt.subplot(2,2,3)
plt.title("Recovered")
plt.imshow(ten2img(img1_r[0]))
plt.axis("off")

plt.subplot(2,2,4)
plt.imshow(ten2img(img2_r[0]))
plt.axis("off")

plt.show()

print(f"Angles Between Them. img1: {angle1.item()}, img2: {angle2.item()}")


### GRP-IoM

In [ ]:
from attacks import attack_grp_efficient

feat3_r = attack_grp_efficient(grp, tmp3)
img3_r = nbnet(F.normalize(feat3_r))
feat3_rr = backbone(F.interpolate(img3_r, (112,112)))
angle3 = F.cosine_similarity(feat3, feat3_rr).acos() * 180 / torch.pi

plt.subplot(1,2,1)
plt.title("Enrolled")
plt.imshow(img3)
plt.axis("off")

plt.subplot(1,2,2)
plt.title("Recovered")
plt.imshow(ten2img(img3_r[0]))
plt.axis("off")

print("Angle Between Them: ", angle3.item())

plt.show()

### URP-IoM

In [ ]:
from attacks import attack_LSH_anom, loss_fn_urp
feat4_r = attack_LSH_anom(tmp4, loss_fn_urp, urp.w)
feat5_r = attack_LSH_anom(tmp5, loss_fn_urp, urp.w)

img4_r = nbnet(F.normalize(feat4_r))
img5_r = nbnet(F.normalize(feat5_r))

feat4_rr = backbone(F.interpolate(img4_r, (112,112)))
feat5_rr = backbone(F.interpolate(img5_r, (112,112)))

angle4 = F.cosine_similarity(feat4, feat4_rr).acos() * 180 / torch.pi
angle5 = F.cosine_similarity(feat5, feat5_rr).acos() * 180 / torch.pi

if (angle4 > 90):
    feat4_r = -feat4_r
    
if (angle5 > 90):
    feat5_r = -feat5_r


img4_r = nbnet(F.normalize(feat4_r))
img5_r = nbnet(F.normalize(feat5_r))

feat4_rr = backbone(F.interpolate(img4_r, (112,112)))
feat5_rr = backbone(F.interpolate(img5_r, (112,112)))

angle4 = F.cosine_similarity(feat4, feat4_rr).acos() * 180 / torch.pi
angle5 = F.cosine_similarity(feat5, feat5_rr).acos() * 180 / torch.pi

    
    
plt.subplot(2,2,1)
plt.title("Enrolled")
plt.imshow(img4)
plt.axis("off")

plt.subplot(2,2,2)
plt.imshow(img5)
plt.axis("off")

plt.subplot(2,2,3)
plt.title("Recovered")
plt.imshow(ten2img(img4_r[0]))
plt.axis("off")

plt.subplot(2,2,4)
plt.imshow(ten2img(img5_r[0]))
plt.axis("off")

plt.show()

print(f"Angles Between Them. img4: {angle4.item()}, img5: {angle5.item()}")



## 3. Comparison with Previous Attacks

### Dong et al.

In [ ]:
from attacks import attack_GA
import numpy as np

feat1_ga = attack_GA(abh, tmp1)
feat2_ga = attack_GA(abh, tmp2)
feat3_ga = attack_GA(grp, tmp3)
feat4_ga = attack_GA(urp, tmp4)

img1_ga = nbnet(F.normalize(feat1_ga))
img2_ga = nbnet(F.normalize(feat2_ga))
img3_ga = nbnet(F.normalize(feat3_ga))
img4_ga = nbnet(F.normalize(feat4_ga))

for i in range(1, 13):
    if (i-1)//4 == 0:
        img = globals()["img" + str(i)]
    
    elif (i-1)//4 == 1:
        img = globals()["img" + str(i-4) + "_ga"]
    
    else:
        img = globals()["img" + str(i-8) + "_r"]
    
    plt.subplot(3, 4, i)
    if isinstance(img, np.ndarray):
        plt.imshow(img)
    else:
        plt.imshow(ten2img(img[0]))
    plt.axis("off")

plt.show()


### Ghammam et al.

In [ ]:
from attacks import attack_urp_efficient

feat5_gham = attack_urp_efficient(urp, tmp5)
img5_gham = nbnet(F.normalize(feat5_gham))

feat5_rgham = backbone(F.interpolate(img5_gham, (112,112)))
angle5_gham = F.cosine_similarity(feat5, feat5_rgham).acos() * 180 / torch.pi

plt.subplot(1,3,1)
plt.title("Enrolled")
plt.imshow(img5)
plt.axis("off")

plt.subplot(1,3,2)
plt.title("Recovered by \n Ghammam et al.")
plt.imshow(ten2img(img5_gham[0]))
plt.axis("off")

plt.subplot(1,3,3)
plt.title(r"Recovered by Ours.")
plt.imshow(ten2img(img5_r[0]))
plt.axis("off")

plt.show()

print("Angle Between Them (Ghammam): ", angle5_gham.item())
print("Angle Between Them (Ours): ", angle5.item())

